In [1]:
import pandas as pd
import numpy as np
import holidays
from datetime import datetime
import re
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

# Data cleaning, imputation, and encoding


**Authors:** Tobias Schulze, Tiago Russomano, Johanna Stärkl

**Last update:** 16 October 2023

All imported files were preprocessed to standardize file name schemes and separators.

In general, issues occurring during import and data type conversion was fixed on the fly.

Further curation will be performed after in deep evaluation of the data.

## Goals of this notebook

In [2]:
df = pd.read_csv('./data/230819_input_table.csv', low_memory=False, header = 0, index_col=0, na_values='n/a')

In [3]:
# store date in string format
df['date'] = df.apply(lambda row: datetime(row['an'], row['mois'], row['jour']).strftime('%Y-%m-%d'), axis = 1)
df.head()

,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,...,env1,senc,catv,occutc,obs,obsm,choc,manv,motor,date
Num_Acc,,,,,,,,,,,,,,,,,,,,,
201900000001,138306524.0,B01,2.0,2,4,2,2002.0,0.0,1.0,0.0,...,NaN,2.0,7,NaN,0.0,2.0,5.0,23.0,1.0,2019-11-30
201900000001,138306524.0,B01,1.0,1,4,2,1993.0,5.0,1.0,0.0,...,NaN,2.0,7,NaN,0.0,2.0,5.0,23.0,1.0,2019-11-30
201900000001,138306525.0,A01,1.0,1,1,1,1959.0,0.0,1.0,0.0,...,NaN,2.0,17,NaN,1.0,0.0,3.0,11.0,1.0,2019-11-30
201900000002,138306523.0,A01,1.0,1,4,2,1994.0,0.0,1.0,0.0,...,NaN,1.0,7,NaN,4.0,0.0,1.0,0.0,1.0,2019-11-30
201900000003,138306520.0,A01,1.0,1,1,1,1996.0,0.0,1.0,0.0,...,NaN,1.0,7,NaN,0.0,2.0,1.0,2.0,1.0,2019-11-28


In [4]:
# get holiday list for France
fr_holidays = holidays.FR()
df['is_holiday'] = df.apply(lambda row: row['date'] in fr_holidays, axis = 1)
df['holiday'] = df.apply(lambda row: fr_holidays.get(row['date']) if row['is_holiday'] == True else np.nan, axis = 1)
df.head()

,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,...,catv,occutc,obs,obsm,choc,manv,motor,date,is_holiday,holiday
Num_Acc,,,,,,,,,,,,,,,,,,,,,
201900000001,138306524.0,B01,2.0,2,4,2,2002.0,0.0,1.0,0.0,...,7,NaN,0.0,2.0,5.0,23.0,1.0,2019-11-30,False,NaN
201900000001,138306524.0,B01,1.0,1,4,2,1993.0,5.0,1.0,0.0,...,7,NaN,0.0,2.0,5.0,23.0,1.0,2019-11-30,False,NaN
201900000001,138306525.0,A01,1.0,1,1,1,1959.0,0.0,1.0,0.0,...,17,NaN,1.0,0.0,3.0,11.0,1.0,2019-11-30,False,NaN
201900000002,138306523.0,A01,1.0,1,4,2,1994.0,0.0,1.0,0.0,...,7,NaN,4.0,0.0,1.0,0.0,1.0,2019-11-30,False,NaN
201900000003,138306520.0,A01,1.0,1,1,1,1996.0,0.0,1.0,0.0,...,7,NaN,0.0,2.0,1.0,2.0,1.0,2019-11-28,False,NaN


In [5]:
df['holiday'].unique()

array([nan, 'Ascension Day', 'Victory Day', 'Labor Day', 'Assumption Day',
       'Whit Monday', "New Year's Day", 'Easter Monday', 'Christmas Day',
       'Armistice Day', 'National Day', "All Saints' Day",
       'Ascension Day; Labor Day'], dtype=object)

In [6]:
#calculate missing values of each colum
percent_missing = round(df.isnull().sum() * 100 / len(df), 1)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
percent_missing

id_vehicule     85.4
num_veh          0.0
place            4.9
catu             0.0
grav             0.0
sexe             0.0
an_nais          0.2
trajet           0.0
secu1           85.4
secu2           85.4
secu3           85.4
locp             2.2
actp             2.2
etatp            2.2
secu            16.9
an               0.0
mois             0.0
jour             0.0
hrmn             0.0
lum              0.0
agg              0.0
int              0.0
atm              0.0
col              0.0
com              0.0
adr             13.7
gps             56.3
lat             42.5
long            42.8
dep              0.0
metropolitan     0.0
catr             0.0
voie             9.0
v1              56.4
v2              95.2
circ             0.1
nbv              0.2
vosp             0.3
prof             0.2
pr              40.7
pr1             40.8
plan             0.2
lartpc          19.7
larrout         10.2
surf             0.2
infra            0.5
situ             0.5
vma          

In [7]:
#cleaning of the dataset regarding missing values

#delete columns id_vehicule, com, gps, voie, v1, v2, pr, pr1, lartpc, larrout, vma, env1, motor, secu1, 
#secu2, secu3, secu

df.drop(['id_vehicule', 'com', 'gps', 'voie', 'v1', 'v2',
         'pr', 'pr1', 'lartpc', 'larrout', 'vma', 'env1', 'motor', 
         'secu', 'secu1', 'secu2', 'secu3'], axis=1, inplace=True)


In [8]:
#replace NaN with -1 (coded as no information) in columns
#place, an_nais, trajet, locp, actp, etatp ,atm, col, circ, nbv, vosp, prof, plan, surf, infra
#situ, senc, occutc, obs, obsm, choc, manv

df.fillna({'place':-1, 'an_nais':-1, 'trajet':-1, 'locp':-1, 'actp': -1,
           'etatp':-1, 'atm':-1, 'col':-1, 'circ':-1, 
           'nbv':-1, 'vosp':-1, 'prof':-1, 'plan': -1, 
           'surf':-1, 'infra':-1, 'situ': -1, 'senc':-1, 'occutc':-1, 
           'obs': -1, 'obsm': -1, 'choc':-1, 'manv':-1}, inplace=True)

In [9]:
#remove lines with NaN in column catr

df.dropna(subset=['catr'], inplace=True)


In [10]:
#transform date to datetime
df['date'] =  pd.to_datetime(df['date'], format="%Y-%m-%d")

df['date'] = df['date'].dt.date

df['date'].describe()

count        2509596
unique          6209
top       2005-12-16
freq             832
Name: date, dtype: object

In [11]:
percent_missing_clean = round(df.isnull().sum() * 100 / len(df), 1)
percent_missing_clean

num_veh          0.0
place            0.0
catu             0.0
grav             0.0
sexe             0.0
an_nais          0.0
trajet           0.0
locp             0.0
actp             0.0
etatp            0.0
an               0.0
mois             0.0
jour             0.0
hrmn             0.0
lum              0.0
agg              0.0
int              0.0
atm              0.0
col              0.0
adr             13.7
lat             42.5
long            42.8
dep              0.0
metropolitan     0.0
catr             0.0
circ             0.0
nbv              0.0
vosp             0.0
prof             0.0
plan             0.0
surf             0.0
infra            0.0
situ             0.0
senc             0.0
catv             0.0
occutc           0.0
obs              0.0
obsm             0.0
choc             0.0
manv             0.0
date             0.0
is_holiday       0.0
holiday         97.7
dtype: float64

## Transform data types
### Date and time variables
The date and time variables are maybe important as grouping variables or as contrains for time dependent severity of accidents.

For the grouping, a timestamp is required for unbiased identification time related accidents.

_Problem_:

During the transformation of the `hrmn` variable, I got aware, that the string contain integers like `1`,  `801`, or `1300`. Hence, anytime during data conversion, the colon got lost and the values got truncated. Hence `1` should be `00:01` and so on. Therefore, we need an additional transformation of the truncated data to `hh:mm` format.

**Steps:**
1. Fixing the truncated values in `hrmn`

~~3. Creation of a `datatime` variable in format y-m-d hh:mm~~

~~4. Transformation of the datatime varible to a `timestamp` variable~~


In addition, we need to transform the type of `an_nais` to integer and `date` to `date`.

In [12]:
# Fixing the hrmn issue:
# Remove the colon
df['hrmn'] = df.apply(lambda x: re.sub(string=x['hrmn'], pattern=':', repl=''), axis = 1)

# Pad the string to four zeros
df['hrmn'] = df.apply(lambda x: x['hrmn'].zfill(4), axis = 1)

# Transform the variable to 'hh:mm' and split to hours and minutes
df = df.assign(hrmn = pd.to_datetime(df['hrmn'], format='%H%M').dt.strftime('%H:%M'))

# Create the daytime variable
#df['datetime'] = df.apply(lambda x: datetime(x['an'], x['mois'], x['jour'], datetime.strptime(x['hrmn'], "%H:%M").hour, datetime.strptime(x['hrmn'], "%H:%M").minute), axis = 1)

# Create the timestamp
#df['timestamp'] = df.apply(lambda x: datetime.timestamp(x['datetime']), axis = 1)

# Transform `an_nais`
df['an_nais'] = df['an_nais'].astype('int64')

# Transform `date`
df = df.assign(date = pd.to_datetime(df['date'], format='mixed'))

## Filtering
### Drop non-metropolitan departments
It was decided to use only accidents in metropolitan France and Corse.

In preprocessing, the varible `metropolitan` with values `[0,1]` was created.

Now, the data is fitered by this variable and then it is dropped.

In [13]:
df = df.loc[df['metropolitan'] == 1]
df.drop('metropolitan', inplace=True, axis=1)

In [14]:
drop_columns = ['adr', 'lat', 'long']
df.drop(drop_columns, inplace = True, axis=1)

### Clean gravity
Gravity still contains data expressed by `-1` which is related to unknown injury. We need to remove this data.

In [15]:
unknown_count = (df['grav'] == -1).sum()

# Print the count
print(f"Number of data points with the value -1 in the 'grav' variable: {unknown_count}")

Number of data points with the value -1 in the 'grav' variable: 54


In [16]:
df = df[df['grav'] != -1]
df.isna().sum()

num_veh             0
place               0
catu                0
grav                0
sexe                0
an_nais             0
trajet              0
locp                0
actp                0
etatp               0
an                  0
mois                0
jour                0
hrmn                0
lum                 0
agg                 0
int                 0
atm                 0
col                 0
dep                 0
catr                0
circ                0
nbv                 0
vosp                0
prof                0
plan                0
surf                0
infra               0
situ                0
senc                0
catv                0
occutc              0
obs                 0
obsm                0
choc                0
manv                0
date                0
is_holiday          0
holiday       2367768
dtype: int64

### Encoding
For the first trial, use the `LabelEncoder` to encode categorial values. Then drop the old categorial values and replace them by the encoded.

In [18]:
# Encoding with LabelEncoder
encode_columns = ['actp', 'num_veh', 'hrmn', 'grav', 'etatp']
encoded_df = df[encode_columns]
encoded_df = encoded_df.astype('str')
encoded_df = encoded_df.apply(LabelEncoder().fit_transform)

# Merge encoded values
df_encoded = df
df_encoded.drop(encode_columns, inplace=True, axis=1)
df_encoded = pd.concat([encoded_df, df_encoded], axis=1)


In [20]:
df_encoded.to_csv("./data/231017_clean_table_for_analysis.csv", sep = ',', header = True, na_rep = 'n/a', index=True)